**EPTlib** is an open-source C++ library implementing Electric Properties Tomography (EPT) methods.
- You can download it from the website https://eptlib.github.io.
  - The sources can be compiled using CMake, having installed the libraries listed in the ReadMe.
  - Only for Windows, you can also download a binary installer.

The library comes with a terminal application that allows applying the implemented EPT methods directly on input data.
- In this course you will learn how to use this terminal application.
- Input pre-processing, configuration settings and output post-processing, as well as the application execution, will be done using Python.
  - You can download and install Python with all the packages needed for this tutorial from https://www.anaconda.com/.

To start, we just need to move the installed or compiled `bin` folder of EPTlib in the current directory.

# Let's start

First of all, we need to import the Python packages that will be used in the following:
- `h5py` is used for writing and reading the input/output files of EPTlib;
- `matplotlib` allows to generate plots and pictures from the data;
- `numpy` is a library for handling vectors and matrices and implements useful numerical functions;
- `os` allows to interact with the Operative System;
- `skimage` is a library for image processing;
- `toml` is used for writing the configuration file of EPTlib.

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage.morphology as morpho
import toml

The file `15t-brain.h5` in the folder `data` contains the input for our model problem. They have been obtained by simulating the head of *subject04* from the [BrainWeb database](http://www.bic.mni.mcgill.ca/brainweb/) irradiated with a 1.5 T birdcage coil.

Precisely, four 3D arrays are available representing data acquired in a human head:
- `tx-sens` is the map of the transmit sensitivity $|B_1^+|$;
- `trx-phase` is the map of the transceive phase $\varphi^\pm$;
- `mask-wm` is a (perfect) segmentation of the white matter;
- `sigma` is the reference map of the electrical conductivity in S/m.

Let's read the maps and see how they appear.

In [ ]:
# read the maps
address = os.path.join("data","15t-brain.h5")
with h5py.File(address,"r") as ifile:
    tx_sens = ifile["tx-sens"][()]
    trx_phase = ifile["trx-phase"][()]
    mask_wm = ifile["mask-wm"][()]
    sigma = ifile["sigma"][()]
# store the map sizes
nn = tx_sens.shape
# plot the maps in the mid-plane
def plot_midplane(map,nn,title):
    plt.imshow(map[nn[0]//2,:,:])
    plt.colorbar()
    plt.title(title)
plt.figure()
plt.subplot(2,2,1)
plot_midplane(tx_sens,nn,"Tx sens")
plt.subplot(2,2,2)
plot_midplane(trx_phase,nn,"TRx phase (rad)")
plt.subplot(2,2,3)
plot_midplane(mask_wm,nn,"WM mask")
plt.subplot(2,2,4)
plot_midplane(sigma,nn,"Sigma (S/m)")
plt.tight_layout()
plt.show()

The maps stored in `tx_sens` and `trx_phase` are obtained from accurate numerical simulations and are noiseless.

In order to simulate a more realistic problem, we can corrupt the maps with an additive Gaussian noise.

In [ ]:
snr = 100
# compute the standard deviation
std = np.nanmean(tx_sens)/snr
# apply the noise to the complex map
img = tx_sens*np.exp(1j*trx_phase)
img_n = img+np.random.normal(scale=std,size=nn)+1j*np.random.normal(scale=std,size=nn)
# extract the noisy maps
tx_sens_n = np.abs(img_n)
trx_phase_n = np.angle(img_n)
# plot the maps in the mid-plane
plt.figure()
plt.subplot(1,2,1)
plot_midplane(tx_sens_n,nn,"Tx sens")
plt.subplot(1,2,2)
plot_midplane(trx_phase_n,nn,"TRx phase")
plt.tight_layout()
plt.show()

# Set-up the configuration file
The terminal application of EPTlib is configured by a .toml file.

A number of settings is independent of the used EPT method, whereas other settings are specific for each method. Each setting is a value (boolean, numerical, or a string) associated to a key string. With the Python package `toml` we can automatically produce the .toml file in accordance to a Python dictionary.

We start defining a function for the common settings.

In [ ]:
def create_table_conditionally(config, label):
    if (label not in config.keys() or not isinstance(config[label],dict)):
        config[label] = {}
def config_common_settings(config, title,description,size,step,freq):
    config["title"] = title
    config["description"] = description
    create_table_conditionally(config,"mesh")
    config["mesh"]["size"] = size
    config["mesh"]["step"] = step
    create_table_conditionally(config,"input")
    config["input"]["frequency"] = freq
    config["input"]["tx-channels"] = 1
    config["input"]["rx-channels"] = 1
    config["input"]["wrapped-phase"] = False
# define the setting values
title = "EPTlib tutorial"
description = "Let's learn how to use EPTlib through Python"
size = nn
step = [2e-3,2e-3,2e-3] # [m]
freq = 64e6 # [Hz]
# initialize the dictionary and fill it
config = {}
config_common_settings(config, title,description,size,step,freq)
# save the settings in a .toml file
address = os.path.join("settings","example.toml")
with open(address,"w") as ofile:
    toml.dump(config,ofile)

Then, we need to decide which are the input and the output images, which are provided in the HDF5 format. So, we have to store our input data in an `.h5` file, which will contain both the noiseless and the corrupted images.

**N.B.** The Matlab `.mat` file with format version v7.3 is an HDF5 file, so its datasets can be accessed directly by EPTlib!

It is important to note that the selection of the input images in the configuration file has consequences on the adopted EPT method. In particular, having both the Tx sensitivity and the TRx phase will lead to the application of a complex-valued method. Setting up only the TRx phase (or the Tx sensitivity), instead, will lead to a phase-based (or magnitude-based) approximation of the selected method, if available.

In [ ]:
# write the HDF5 file with noiseless and noisy input
address = os.path.join("settings","15t-head-input.h5")
with h5py.File(address,"w") as ofile:
    group_noiseless = ofile.create_group("noiseless")
    group_noiseless.create_dataset("tx-sens",data=tx_sens)
    group_noiseless.create_dataset("trx-phase",data=trx_phase)
    group_noisy = ofile.create_group("snr{:d}".format(snr))
    group_noisy.create_dataset("tx-sens",data=tx_sens_n)
    group_noisy.create_dataset("trx-phase",data=trx_phase_n)
# store the image addresses in strings
addr_noiseless_tx_sens = address+":/"+"noiseless"+"/"+"tx-sens"
addr_noiseless_trx_phase = address+":/"+"noiseless"+"/"+"trx-phase"
addr_noisy_tx_sens = address+":/"+"snr{:d}".format(snr)+"/"+"tx-sens"
addr_noisy_trx_phase = address+":/"+"snr{:d}".format(snr)+"/"+"trx-phase"
# initialize the name of the output file
ofname = "15t-head-output.h5"
# define a function configuring I/O settings
def config_io_settings(config, in_tx_sens,in_trx_phase,out_sigma,out_epsr):
    create_table_conditionally(config,"input")
    config["input"]["tx-sensitivity"] = in_tx_sens
    config["input"]["trx-phase"] = in_trx_phase
    create_table_conditionally(config,"output")
    config["output"]["electric-conductivity"] = out_sigma
    config["output"]["relative-permittivity"] = out_epsr

The EPT method is selected by setting the option `method` according to the following table,

| code | method                                                       |
|:----:|:-------------------------------------------------------------|
| 0    | Helmholtz EPT                                                |
| 1    | Convection-reaction EPT                                      |
| 2    | Gradient EPT                                                 |
| 100  | Phase-based Helmholtz EPT with automatically selected kernel |

Starting from code 100, the implemented methods are *experimental*.

In [ ]:
methods = {
    "Helmholtz EPT": 0,
    "Convection-reaction EPT": 1,
    "Gradient EPT": 2
}
# define a function configuring the method setting
def set_option_from_dictionary(config, label,value,dictionary):
    if isinstance(value,str):
        config[label] = dictionary[value]
    else:
        config[label] = value
def config_method_settings(config, method):
    set_option_from_dictionary(config, "method",method,methods)

## Helmholtz-EPT settings

A number of parameters defining the Savitzky-Golay (SG) filter must be set for Helmholtz EPT. The SG filter estimates the spatial derivatives of the input maps by approximating them locally according to a polynomial fitting. The resulting polynomial is then derived analytically.
The parameters are needed to select the degree of the polynomial and to define the concept of *locality* by introducing a kernel with a certain shape and size.

<img src="imgs/kernels.png" alt="Kernels" width="50%"/>

The parameter `shape` is a numerical value identifying the shape of the kernel according to this table,

| code | shape  |
|:----:|:-------|
| 0    | Cross  |
| 1    | Sphere |
| 2    | Cube   |

The parameter `size` is the semi-length in each direction of the kernel bounding box.

In [ ]:
shapes = {
    "Cross": 0,
    "Sphere": 1,
    "Cube": 2
}
# define a function configuring the SG filter
def config_sg_settings(config, size,shape,degree):
    create_table_conditionally(config,"parameter")
    create_table_conditionally(config["parameter"],"savitzky-golay")
    config["parameter"]["savitzky-golay"]["size"] = size
    set_option_from_dictionary(config["parameter"]["savitzky-golay"], "shape",shape,shapes)
    config["parameter"]["savitzky-golay"]["degree"] = degree

Finally, we can create the configuration files for all the combinations of Helmholtz EPT that we would like to compare:
- Phase-based approximation with noiseless input;
- Phase-based approximation with noisy input;
- Complex-valued method with noiseless input;
- Complex-valued method with noisy input.

In order to simplify the creation of all the configuration files and the program executions by the adoption of loops, the configurations themselves can be collected in a dictionary.

In [ ]:

# define a fuction for the combined setup of H-EPT
def config_hept_settings(config, title,description,size,step,freq, method, sg_size,sg_shape,sg_degree, in_tx_sens,in_trx_phase,out_sigma,out_epsr):
    config_common_settings(config, title,description,size,step,freq)
    config_method_settings(config, method)
    config_sg_settings(config, sg_size,sg_shape,sg_degree)
    config_io_settings(config, in_tx_sens,in_trx_phase,out_sigma,out_epsr)
# initialize the common settings
title = "Helmholtz EPT tutorial"
method = "Helmholtz EPT"
size = nn[::-1]
step = [2e-3,2e-3,2e-3]
freq = 64e6
sg_size = [2,2,2]
sg_shape = "Sphere"
sg_degree = 2
# create all the configurations with H-EPT
configurations = {}
# phase-based approximation - noiseless data
key = "hept-phase-noiseless"
description = "Phase-based approximation - Noiseless data"
addr_tx_sens = ""
addr_trx_phase = addr_noiseless_trx_phase
addr_sigma = ofname+":/"+key.replace("-","/")+"/"+"sigma"
addr_epsr = ""
configurations[key] = {}
config_hept_settings(configurations[key], title,description,size,step,freq,method,sg_size,sg_shape,sg_degree,addr_tx_sens,addr_trx_phase,addr_sigma,addr_epsr)
# phase-based approximation - noisy data
key = "hept-phase-snr{:d}".format(snr)
description = "Phase-based approximation - Noisy data"
addr_tx_sens = ""
addr_trx_phase = addr_noisy_trx_phase
addr_sigma = ofname+":/"+key.replace("-","/")+"/"+"sigma"
addr_epsr = ""
configurations[key] = {}
config_hept_settings(configurations[key], title,description,size,step,freq,method,sg_size,sg_shape,sg_degree,addr_tx_sens,addr_trx_phase,addr_sigma,addr_epsr)
# complex-valued method - noiseless data
key = "hept-complex-noiseless"
description = "Complex-valued method - Noiseless data"
addr_tx_sens = addr_noiseless_tx_sens
addr_trx_phase = addr_noiseless_trx_phase
addr_sigma = ofname+":/"+key.replace("-","/")+"/"+"sigma"
addr_epsr = ofname+":/"+key.replace("-","/")+"/"+"epsr"
configurations[key] = {}
config_hept_settings(configurations[key], title,description,size,step,freq,method,sg_size,sg_shape,sg_degree,addr_tx_sens,addr_trx_phase,addr_sigma,addr_epsr)
# complex-valued method - noiseless data
key = "hept-complex-snr{:d}".format(snr)
description = "Complex-valued method - Noisy data"
addr_tx_sens = addr_noisy_tx_sens
addr_trx_phase = addr_noisy_trx_phase
addr_sigma = ofname+":/"+key.replace("-","/")+"/"+"sigma"
addr_epsr = ofname+":/"+key.replace("-","/")+"/"+"epsr"
configurations[key] = {}
config_hept_settings(configurations[key], title,description,size,step,freq,method,sg_size,sg_shape,sg_degree,addr_tx_sens,addr_trx_phase,addr_sigma,addr_epsr)

## Convection-reaction EPT settings

The current complex-valued implementation is really unstable, so we will use only the phase-based approximation!

Convection-reaction EPT requires the settings of the SG filter already seen for Helmholtz EPT.
In addition, parameters for the following properties must the defined:
- Dirichlet boundary conditions on electric conductivity (and relative permittivity, in the complex-valued case);
- Three-dimensional or two-dimensional (approximated) problem;
- Stabilization through artificial diffusion.

In [ ]:
# define a function configuring the boundary conditions
def config_bc_settings(config, sigma,epsr):
    create_table_conditionally(config,"parameter")
    create_table_conditionally(config["parameter"],"dirichlet")
    config["parameter"]["dirichlet"]["electric-conductivity"] = sigma
    config["parameter"]["dirichlet"]["relative-permittivity"] = epsr
# define a function configuring the problem dimensions
def config_volume_settings(config, volume_tomography,slice=None):
    create_table_conditionally(config,"parameter")
    config["parameter"]["volume-tomography"] = volume_tomography
    if (not volume_tomography and slice):
        config["parameter"]["imaging-slice"] = slice
# define a function configuring the artificial diffusion
def config_artificial_diffusion_settings(config, coeff):
    create_table_conditionally(config,"parameter")
    if (coeff>0):
        config["parameter"]["artificial-diffusion"] = True
        config["parameter"]["artificial-diffusion-coefficient"] = coeff

Now, we have the tools to create the new configurations for Convection-reaction EPT in the following conditions:
- Phase-based approximation with noiseless input;
- Phase-based approximation with noisy input.

In [ ]:
# define a fuction for the combined setup of CR-EPT
def config_crept_settings(config, title,description,size,step,freq, method, sg_size,sg_shape,sg_degree, bc_sigma,bc_epsr, volume_tomography, ad_coeff, in_tx_sens,in_trx_phase,out_sigma,out_epsr):
    config_common_settings(config, title,description,size,step,freq)
    config_method_settings(config, method)
    config_sg_settings(config, sg_size,sg_shape,sg_degree)
    config_bc_settings(config, bc_sigma,bc_epsr)
    config_volume_settings(config, volume_tomography)
    config_artificial_diffusion_settings(config, ad_coeff)
    config_io_settings(config, in_tx_sens,in_trx_phase,out_sigma,out_epsr)
# initialize the common settings
title = "Convection-reaction EPT tutorial"
method = "Convection-reaction EPT"
size = nn[::-1]
step = [2e-3,2e-3,2e-3]
freq = 64e6
sg_size = [2,2,2]
sg_shape = "Sphere"
sg_degree = 2
bc_sigma = 0.3
bc_epsr = 1.0
volume_tomography = True
ad_coeff = 0.01
# phase-based approximation - noiseless data
key = "crept-phase-noiseless"
description = "Phase-based approximation - Noiseless data"
addr_tx_sens = ""
addr_trx_phase = addr_noiseless_trx_phase
addr_sigma = ofname+":/"+key.replace("-","/")+"/"+"sigma"
addr_epsr = ""
configurations[key] = {}
config_crept_settings(configurations[key], title,description,size,step,freq,method,sg_size,sg_shape,sg_degree,bc_sigma,bc_epsr,volume_tomography,ad_coeff,addr_tx_sens,addr_trx_phase,addr_sigma,addr_epsr)
# phase-based approximation - noisy data
key = "crept-phase-snr{:d}".format(snr)
description = "Phase-based approximation - Noisy data"
addr_tx_sens = ""
addr_trx_phase = addr_noisy_trx_phase
addr_sigma = ofname+":/"+key.replace("-","/")+"/"+"sigma"
addr_epsr = ""
configurations[key] = {}
config_crept_settings(configurations[key], title,description,size,step,freq,method,sg_size,sg_shape,sg_degree,bc_sigma,bc_epsr,volume_tomography,ad_coeff,addr_tx_sens,addr_trx_phase,addr_sigma,addr_epsr)

Finally, with a simple loop we can create all the configuration files!

In [ ]:
# create the configuration files
for key in configurations.keys():
    fname = os.path.join("settings",key+".toml")
    with open(fname,"w") as ofile:
        toml.dump(configurations[key],ofile)

# Execute EPTlib

Once the configuration files are ready, to perform the EPT is as simple as launching the EPTlib executable.

In [ ]:
# launch EPTlib
for key in configurations.keys():
    executable = os.path.join("bin","EPTlib_app")
    config = os.path.join("settings",key+".toml")
    log = os.path.join("logs",key+".log")
    command = "{:s} {:s} > {:s}".format(executable,config,log)
    os.system(command)

# Post-process the results

Now we need to load in Python the results of EPTlib. Since the dimensions are small enough, we load all the conductivity maps at the same time and store them in a dictionary.

In [ ]:
# read the conductivity maps
conductivities = {}
for key in configurations.keys():
	addr_sigma = configurations[key]["output"]["electric-conductivity"]
	fname,url = addr_sigma.split(":")
	with h5py.File(fname) as ifile:
		groups = url.split("/")[1:]
		ptr = ifile
		for group in groups:
			ptr = ptr[group]
		conductivities[key] = ptr[()]

Once the maps have been loaded, first of all we can plot them for a visual inspection of the results.

In [ ]:
# plot the conductivity maps in the mid-plane
plt.figure(figsize=(6,9))

plt.subplot(3,2,1)
plt.ylabel("H-EPT complex")
plot_midplane(conductivities["hept-complex-noiseless"],nn,"Noiseless")
plt.clim((0,2))
plt.subplot(3,2,2)
plot_midplane(conductivities["hept-complex-snr{:d}".format(snr)],nn,"SNR={:d}".format(snr))
plt.clim((0,2))

plt.subplot(3,2,3)
plt.ylabel("H-EPT phase")
plot_midplane(conductivities["hept-phase-noiseless"],nn,"")
plt.clim((0,2))
plt.subplot(3,2,4)
plot_midplane(conductivities["hept-phase-snr{:d}".format(snr)],nn,"")
plt.clim((0,2))

plt.subplot(3,2,5)
plt.ylabel("CR-EPT phase")
plot_midplane(conductivities["crept-phase-noiseless"],nn,"")
plt.clim((0,2))
plt.subplot(3,2,6)
plot_midplane(conductivities["crept-phase-snr{:d}".format(snr)],nn,"")
plt.clim((0,2))

plt.tight_layout()
plt.show()

The reconstruction with SNR 100 is extremely noisy for Helmholtz EPT with respect to Convection-reaction EPT. On the other hand, the latter even appears to be over-regularized.
We could change the parameters defined above (SG kernel size and shape, artificial diffusion coefficient) until the obtained maps are satisfactory. We could even automatize such an optimization in a larger Python code.
However, for now, we will keep these results and look for some *integral* post-processing analysis.

Let's focus on the white matter, of which we have a (perfect) segmentation. We want to compare the retrieved average electrical conductivity values in the white matter with the expected one.

In [ ]:
# define a function for computing the averages in a certain region
def averages(mask):
	# extract the actual conductivity of white matter
	sigma_actual = np.mean(sigma[mask])
	print("Actual conductivity value (S/m): {:.3f}".format(sigma_actual))
	# extract the estimated conductivities of white matter
	sigma_avg = {}
	sigma_std = {}
	for key in conductivities.keys():
		sigma_avg[key] = np.nanmean(conductivities[key][mask])
		sigma_std[key] = np.nanstd(conductivities[key][mask])
		print("Conductivity value (S/m): [{:s}] {:.3f} +- {:.3f}".format(key,sigma_avg[key],sigma_std[key]))
# compute the averages in the white matter
averages(mask_wm)

It appears that the conductivity is strongly over-estimated, especially by Helmholtz EPT.
This can be due to different reasons:
- Usually, phase-based approximations slightly over-estimates the conductivity values;
- There are boundary effects that are particularly strong in Helmholtz EPT, but not completely neglibile neither in Convection-reaction EPT.

Since, here, also the complex-valued Helmholtz EPT over-estimates the conductivity value, the first option cannot be assumed as a comprehensive motivation.

To check if the boundary effect has a role in this over-estimation, let's see what happen if we erode the white matter segmentation in order to get a smaller mask.

In [ ]:
# erode the mask with the same kernel used in SG filter
selem = morpho.ball(2)
mask_wm_eroded = morpho.binary_erosion(mask_wm,selem)
# plot the eroded mask for comparison
plt.figure()
plt.subplot(1,2,1)
plot_midplane(mask_wm,nn,"WM mask")
plt.subplot(1,2,2)
plot_midplane(mask_wm_eroded,nn,"eroded")
plt.show()
# compute the averages in the eroded mask
averages(mask_wm_eroded)

These results confirm the boundary effect in the over-estimation of the conductivity value of the white matter.
Moreover, it is interesting to observe that, despite the larger spatial dispersion (with consequent large uncertainty), the averaged values retrieved by Helmholtz EPT in the noisy cases are good approximations of the actual value!

# Concluding remarks

The analysis performed in this tutorial by employing Python scripts can be repeated with any other scripting language you are comfortable with: do not focus on the use of Python, EPTlib is a flexible tool!

If you look for more information on the implemented techniques and the configurable parameters, visit the project website https://eptlib.github.io.